In [38]:
import pandas as pd
import numpy as np

In [39]:
flights_df = pd.read_excel('schedule.xlsx')
flights_df.head()

,FLIGHT_DATE,FLIGHT_TIME,TIME_OF_DAY,AIRLINE_CD,FLIGHT_NO,DEPARTURE_STATION_CD,ARRIVAL_STATION_CD,ARRIVAL_COUNTRY,ARRIVAL_REGION,HAUL,AIRCRAFT_TYPE,FIRST_CLASS_SEATS,BUSINESS_CLASS_SEATS,ECONOMY_SEATS,TIER1_ELIGIBLE_PAX,TIER2_ELIGIBLE_PAX,TIER3_ELIGIBLE_PAX
0,2025-09-02,14:19:00,Afternoon,BA,BA5211,LHR,LAX,USA,North America,LONG,B777,8,49,178,0,10,38
1,2025-06-10,06:42:00,Morning,BA,BA7282,LHR,LAX,USA,North America,LONG,B777,8,49,178,0,7,28
2,2025-10-27,15:33:00,Afternoon,BA,BA1896,LHR,FRA,Germany,Europe,SHORT,A320,0,17,163,0,11,40
3,2025-06-15,18:29:00,Evening,BA,BA5497,LHR,IST,Turkey,Europe,SHORT,A320,0,8,172,0,16,54
4,2025-08-25,20:35:00,Evening,BA,BA1493,LHR,FRA,Germany,Europe,SHORT,A320,0,13,167,0,6,27


In [ ]:
# Categorise flights into long-haul and short-haul then by time of day
long_haul_df = flights_df[flights_df['HAUL'] == 'LONG']
short_haul_df = flights_df[flights_df['HAUL'] == 'SHORT']

morning_df = flights_df[flights_df['TIME_OF_DAY'] == 'Morning']
afternoon_df = flights_df[flights_df['TIME_OF_DAY'] == 'Afternoon']
evening_df = flights_df[flights_df['TIME_OF_DAY'] == 'Evening']

long_morning_df = long_haul_df[long_haul_df['TIME_OF_DAY'] == 'Morning']
long_afternoon_df = long_haul_df[long_haul_df['TIME_OF_DAY'] == 'Afternoon']
long_evening_df = long_haul_df[long_haul_df['TIME_OF_DAY'] == 'Evening']

short_morning_df = short_haul_df[short_haul_df['TIME_OF_DAY'] == 'Morning']
short_afternoon_df = short_haul_df[short_haul_df['TIME_OF_DAY'] == 'Afternoon']
short_evening_df = short_haul_df[short_haul_df['TIME_OF_DAY'] == 'Evening']

In [ ]:
def get_percentages(df):
    tier_1_percentages = []
    tier_3_percentages = []
    
    # Get the average ratio of tier 2 to tier 3 eligible passengers
    tier_2_3_ratio = df['TIER2_ELIGIBLE_PAX'].sum() / df['TIER3_ELIGIBLE_PAX'].sum()

    # Iterate through each row to calculate the percentage of seats in each class
    for index, row in df.iterrows():
        num_first_class = row['FIRST_CLASS_SEATS']
        num_business_class = row['BUSINESS_CLASS_SEATS']
        num_economy_class = row['ECONOMY_SEATS']

        num_tier1_eligble = row['TIER1_ELIGIBLE_PAX']
        num_tier3_eligble = row['TIER3_ELIGIBLE_PAX']

        # Compute an upper bound of the number of tier1 and tier 3 passengers
        tier1_upperbound = num_first_class + num_tier1_eligble
        tier3_upperbound = num_business_class + num_tier3_eligble # Assumes that tier 1 passengers would only go to the tier 1 lounges

        total_seats = num_first_class + num_business_class + num_economy_class

        tier_1_percent = (tier1_upperbound / total_seats) * 100
        tier_1_percentages.append(tier_1_percent)

        tier_3_percent = (tier3_upperbound / total_seats) * 100
        tier_3_percentages.append(tier_3_percent)

    percentile_tier1 = np.percentile(tier_1_percentages, 90)
    percentile_tier3 = np.percentile(tier_3_percentages, 75) 
    tier_2_percentage = percentile_tier3 * tier_2_3_ratio

    return percentile_tier1, tier_2_percentage, percentile_tier3
    
    

In [69]:
short_morning = get_percentages(short_morning_df)
long_morning = get_percentages(long_morning_df)

short_afternoon = get_percentages(short_afternoon_df)
long_afternoon = get_percentages(long_afternoon_df)

short_evening = get_percentages(short_evening_df)
long_evening = get_percentages(long_evening_df)


print("Short Haul Morning (Tier 1, Tier 2, Tier 3): ", short_morning)
print("Long Haul Morning (Tier 1, Tier 2, Tier 3): ", long_morning)

print("Short Haul Afternoon (Tier 1, Tier 2, Tier 3): ", short_afternoon)
print("Long Haul Afternoon (Tier 1, Tier 2, Tier 3): ", long_afternoon)

print("Short Haul Evening (Tier 1, Tier 2, Tier 3): ", short_evening)
print("Long Haul Evening (Tier 1, Tier 2, Tier 3): ", long_evening)


Short Haul Morning (Tier 1, Tier 2, Tier 3):  (1.1111111111111112, 7.374827597267216, 28.333333333333332)
Long Haul Morning (Tier 1, Tier 2, Tier 3):  (3.829787234042553, 8.594598195270324, 32.76595744680851)
Short Haul Afternoon (Tier 1, Tier 2, Tier 3):  (1.1111111111111112, 7.367986596379388, 28.333333333333332)
Long Haul Afternoon (Tier 1, Tier 2, Tier 3):  (3.7037037037037033, 8.703898626205557, 33.11330530259251)
Short Haul Evening (Tier 1, Tier 2, Tier 3):  (1.1111111111111112, 7.421228565492123, 28.333333333333332)
Long Haul Evening (Tier 1, Tier 2, Tier 3):  (3.829787234042553, 8.525193204279239, 32.71801126992853)
